以下是**第二阶段：Python re模块实战**的硬核详解，包含函数解析、分组技巧和性能优化，直接上代码实战👇

---

### **第二阶段：Python re模块实战**
#### **1. 核心函数深度解析**


In [5]:
import re
text = "Contact: service-user.one+@domain.com, phone: 400-800-1234"



| 函数 | 作用 | 典型场景 | 代码示例 | 返回值 |
|------|------|----------|----------|--------|
| **`re.match()`** | **从字符串开头匹配** | 验证格式 | `re.match(r"^Contact", text)` | 匹配对象（成功）或None |
| **`re.search()`** | **扫描整个字符串找第一个匹配** | 提取首个关键信息 | `re.search(r"\d{3}-\d{3}-\d{4}", text)` | 匹配对象（含电话号码） |
| **`re.findall()`** | **返回所有匹配的字符串列表** | 批量提取数据 | `re.findall(r"\w+@\w+\.\w+", text)` | `['service@domain.com']` |
| **`re.finditer()`** | **返回匹配对象的迭代器** | 处理大文本时节省内存 | `for match in re.finditer(r"\d+", text): print(match.group())` | 迭代输出 `400`, `800`, `1234` |
| **`re.sub()`** | **替换匹配内容** | 数据脱敏/格式化 | `re.sub(r"\d", "#", text)` | "Contact: service@domain.com, phone: ###-###-####" |

> 💡 **关键区别**：  
> - `match`只在**行首**匹配，`search`扫描**整个字符串**  
> - `findall`返回字符串列表，`finditer`返回匹配对象（可获取位置）

---

#### **2. 编译正则表达式（re.compile）**
**为什么编译？**  
- 重复使用同一正则时**提升性能**（减少重复解析）  
- 代码更清晰（分离模式定义与使用）



In [ ]:
# 编译一个匹配邮箱的正则
email_pattern = re.compile(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b")
# 复用编译后的对象
emails = email_pattern.findall(text)   # 直接调用方法
print(emails)

['service-user.one+@domain.com']



---

#### **3. 分组捕获 - 提取子内容**

在正则表达式中，**分组捕获**是一个非常实用的功能，它允许你从匹配的文本中**提取特定的子内容**（而不是整个匹配结果）。通过用 `()` 包裹正则的一部分，就能创建一个"分组"，之后可以单独获取这个分组匹配的内容。


### 基本用法：用 `()` 创建分组
- 语法：`(表达式)` —— 括号内的部分会被视为一个分组
- 作用：除了整体匹配外，还会单独记录该分组匹配的内容

##### 3.11 示例1：提取邮箱地址中的用户名和域名
假设我们想从邮箱 `user@example.com` 中分别提取 **用户名（user）** 和 **域名（example.com）**：

In [10]:
email = "user@example.com"
pattern = re.compile(r"(\w+)@(\w.+)")
match =pattern.match(email)
if match:
    print(match.group(0))
    print(match.group(1))
    print(match.group(2))

user@example.com
user
example.com


### 3.12 示例2：提取日期中的年、月、日
从日期字符串 `2023-10-05` 中提取年、月、日：

In [32]:
import re
date = "2025-09-13"
# 三个分组分别对应年、月、日
pattern = re.compile(r"(\d{4})-(\d{2})-(\d{2})")
match = pattern.match(date)
if match:
    print("date:",match.group(0))
    print("年：", match.group(1))  # 2023
    print("月：", match.group(2))  # 10
    print("日：", match.group(3))  # 05

date: 2025-09-13
年： 2025
月： 09
日： 13


#### 关键方法：
- `match.group(n)`：获取第 `n` 个分组的内容（`n=0` 表示整个匹配，`n=1` 开始是第一个分组）
- `match.groups()`：返回所有分组内容的元组（不包含整体匹配）

### 3.13 命名分组：更清晰的提取方式
如果分组较多，用数字（1,2,3）可能容易混淆，可以给分组起名字：

- 语法：`(?P<名称>表达式)` —— 给分组命名
- 提取：`match.group("名称")`


**命名分组** `(?P<name>)` （更易读！）  


In [ ]:
match = re.search(r"(?P<area>\d{3})-(?P<number>\d{3}-\d{4})", text)
if match:
    print(match.group("area"))   # 400
    print(match.groupdict())      # {'area': '400', 'number': '800-1234'}

400
{'area': '400', 'number': '800-1234'}



**非捕获分组** `(?: )` （只分组不捕获）  


In [20]:
# 匹配日期但不捕获分隔符
re.findall(r"(\d{4})(?:-|/)(\d{2})(?:-|/)(\d{2})", "2023-10-01 2023/10/02")
# 输出: [('2023', '10', '01'), ('2023', '10', '02')]

[('2023', '10', '01'), ('2023', '10', '02')]


---

### **⚡ 实战代码实验室**
#### 练习1：数据提取(这里的匹配IP要用一个非贪婪模式，即一旦匹配到IP就停下来)


In [21]:
log = "[ERROR] 2023-10-01 14:30:22 - Connection timeout at 192.168.1.1"
# 提取日志级别、时间戳、IP地址
pattern = re.compile(r"\[(\w+)\] (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) .*? (\d+\.\d+\.\d+\.\d+)")
match = pattern.search(log)

if match:
    print(f"Level: {match.group(1)}, Time: {match.group(2)}, IP: {match.group(3)}")
# 输出: Level: ERROR, Time: 2023-10-01 14:30:22, IP: 192.168.1.1


Level: ERROR, Time: 2023-10-01 14:30:22, IP: 192.168.1.1



#### 练习2：复杂替换


In [27]:
import re
# 将Markdown链接转为HTML标签
md_text = "See [Python文档](https://docs.python.org) 或者 [访问这里]( https://realpython.com)"
html_text = re.sub(
    r"\[(.*?)\]\((.*?)\)", 
    r'<a href="\2">\1</a>', 
    md_text
)
print(html_text)
# 输出: See <a href="https://docs.python.org">document</a> and ...


See <a href="https://docs.python.org">Python文档</a> 或者 <a href=" https://realpython.com">访问这里</a>


### **💡 避坑指南**
1. **`match` vs `search` 陷阱**  

In [23]:
# 失败：match必须从开头匹配
re.match(r"phone", "This phone is")  # None
# 成功：search可匹配任意位置
re.search(r"phone", "This phone is") # 匹配

<re.Match object; span=(5, 10), match='phone'>

1. **贪婪匹配的灾难** 

In [24]:
# 错误：贪婪匹配吞掉整个字符串
re.findall(r"<div>.*</div>", "<div>A</div>...<div>B</div>") 
# 输出: ['<div>A</div>...<div>B</div>'] (一整块)

# 修正：非贪婪模式
re.findall(r"<div>.*?</div>", "<div>A</div>...<div>B</div>")
# 输出: ['<div>A</div>', '<div>B</div>']

['<div>A</div>', '<div>B</div>']

2. **原始字符串强制建议**  

In [ ]:
# 错误：Python会把 \b 解释为退格符
re.findall("\bword\b", text)  # 永远匹配不到！

# 正确：用 r 前缀
re.findall(r"\bword\b", text)

### **📌 第二阶段任务**
1. 用`re.search`+分组提取`"姓名: 张三, 年龄: 30"`中的姓名和年龄  
2. 用`re.sub`将`"价格: $19.99, 折扣: 5%"`中的价格替换为`[PRICE]`，折扣替换为`[DISCOUNT]`  
3. 编写正则匹配并捕获`<img src="image.jpg" alt="logo">`中的src和alt属性值  
4. （挑战）用命名分组解析日志：`"127.0.0.1 - - [01/Oct/2023:15:30:22] GET /index.html"`

---

### **性能提示**


In [31]:
# 在循环中务必预编译！
pattern = re.compile(r"\d+")  # ✅ 高效
big_data_list = "12324342"
for data in big_data_list:
    # 错误：每次循环重新编译 ❌
    # re.findall(r"\d+", data)  
    print(data)
    print("-----")
    # 正确：使用编译后的对象 ✅
    print(pattern.findall(data))
    

1
-----
['1']
2
-----
['2']
3
-----
['3']
2
-----
['2']
4
-----
['4']
3
-----
['3']
4
-----
['4']
2
-----
['2']
